In [ ]:
from moviepy.editor import VideoFileClip, vfx

def mp4_to_gif(input_path, output_path, start_time=None, end_time=None, resize_ratio=1.0, speed=1.0, loop=1, crop_area=None):
    """
    将 MP4 视频转换为 GIF，并支持加速、循环播放和画面裁剪。
    
    参数：
    - input_path: str，输入 MP4 文件路径
    - output_path: str，输出 GIF 文件路径
    - start_time: float，可选，起始时间（秒）
    - end_time: float，可选，结束时间（秒）
    - resize_ratio: float，可选，缩放比例（默认 1.0 不缩放）
    - speed: float，可选，倍速播放（如 2.0 表示加速2倍）
    - loop: int，可选，循环播放次数（默认 1 次）
    - crop_area: tuple，可选，裁剪区域 (x1, y1, x2, y2)，单位像素

    """
    # 读取视频
    clip = VideoFileClip(input_path)
    
    # 裁剪时间
    if start_time is not None and end_time is not None:
        clip = clip.subclip(start_time, end_time)
    
    # 加速播放
    if speed != 1.0:
        clip = clip.fx(vfx.speedx, speed)
    
    # 画面裁剪
    if crop_area is not None:
        x1, y1, x2, y2 = crop_area
        clip = clip.crop(x1=x1, y1=y1, x2=x2, y2=y2)
    
    # 缩放大小
    if resize_ratio != 1.0:
        clip = clip.resize(resize_ratio)
    
    # 循环播放
    clip = clip.loop(n=loop)

    # 保存为 GIF
    clip.write_gif(output_path, fps=10)

# 示例使用
mp4_to_gif(
    input_path="A.mp4", 
    output_path="test.gif", 
    start_time=15, 
    end_time=25, 
    resize_ratio=0.5, 
    speed=2,      # 2 倍速播放
    loop=0,         # 0表示无限循环
    crop_area=(200, 200, 1000, 1000)  # 裁剪画面 (x1, y1, x2, y2)
)


MoviePy - Building file test.gif with imageio.


In [13]:
import cv2
import imageio

def mp4_to_gif(input_path, output_path, start_time=None, end_time=None, resize_ratio=1.0, fps=10, speed_up=2, loop=0):
    """
    将 MP4 转换为 GIF，并支持加速播放和循环设置。

    参数：
    - input_path: 输入 MP4 文件路径
    - output_path: 输出 GIF 文件路径
    - start_time: 截取视频的起始时间（秒）
    - end_time: 截取视频的结束时间（秒）
    - resize_ratio: 缩放比例（默认 1.0，不缩放）
    - fps: GIF 的帧率
    - speed_up: 加速倍数（默认 2 倍，即快放）
    - loop: GIF 循环次数（0 代表无限循环）
    """
    cap = cv2.VideoCapture(input_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)  # 获取原始视频帧率
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 计算起始帧和结束帧
    start_frame = int(start_time * frame_rate) if start_time else 0
    end_frame = int(end_time * frame_rate) if end_time else total_frames

    # 设置视频起始帧
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frames = []
    current_frame = start_frame

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or current_frame >= end_frame:
            break
        

        # 调整尺寸
        if resize_ratio != 1.0:
            frame = cv2.resize(frame, (0, 0), fx=resize_ratio, fy=resize_ratio)

        # 转换 BGR -> RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

        # 控制帧率，调整采样步长，进行加速
        current_frame += (frame_rate // fps) * speed_up
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)

    cap.release()

    # 计算帧间隔（控制播放速度）：默认 1000ms / fps / speed_up
    duration = (1000 / fps) / speed_up

    # 保存 GIF
    imageio.mimsave(output_path, frames, duration=duration / 1000, loop=loop)

# 示例使用：
mp4_to_gif("B.mp4", "5.gif", start_time=115, end_time=125, resize_ratio=0.5, fps=10, speed_up=2, loop=0)
